# BI and Data analytics

## Importing packages

In [1]:
from google.cloud import bigquery
import pandas as pd
import os

## Tasks

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/arman/Downloads/bi-and-data-analytics-task-e40f630391cf.json"

In [3]:
client = bigquery.Client()

### For <span style="color:GREEN">TIME</span> and <span style="color:GREEN">SPACE COMPLEXITY</span> I use <span style="color:GREEN">LIMIT 10</span> in some querries.

### <span style="color:red">1. transactions sent</span>

In [14]:
# All the transactions sent

query_1_1 = """
select  `bigquery-public-data.crypto_ethereum.transactions`.hash as id
from `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status is not null 
limit 10
"""

query_job = client.query(query_1_1)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_1_1 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_1_1.head()

,id
0,0x0860dbefcf40bb92f46e06fc845db68f8f0e3161c762...
1,0xa3db522a11589f55fa2ed160a31bc0cd9ab49073d8c2...
2,0x0449e9b3dd4b67e8a54bff7833ddb70eb81c33ea1833...
3,0x5d9eb6e90cd8f52ebb3bdc90d04f1d7a182a173f5aea...
4,0x6279f7a4b0c2d06221fe813bf31cbc3d9160b5ac6b52...


In [15]:
# The number of transactions sent

query_1_2 = """
select  count(`bigquery-public-data.crypto_ethereum.transactions`.hash) as count
from `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status is not null 
"""

query_job = client.query(query_1_2)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_1_2 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_1_2.head()

,count
0,1515120476


### <span style="color:red">2. transactions recieved 3/ ETH sent</span>

In [16]:
# All the transactions received

query_2_1 = """
select `bigquery-public-data.crypto_ethereum.transactions`.hash as Id
from  `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status = 1
limit 10
"""

query_job = client.query(query_2_1)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_2_1 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_2_1.head()

,Id
0,0x4953aa9ff28bfa2d36a449ab5a01a862ebb62d266ddd...
1,0x721174662b726f696cab200990b56085becfed76dbb3...
2,0xa5b84ac3b6ba8f375fcdb5c021a1dbd79c392250c481...
3,0xf8e887635fbfd58e1700896acf5baa48815824b2594f...
4,0x35dbf4f4581cf564206b28482a7cf346323d4348edf7...


In [17]:
# The number of transactions received

query_2_2 = """
select count(`bigquery-public-data.crypto_ethereum.transactions`.hash) as count
from  `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status = 1
"""

query_job = client.query(query_2_2)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_2_2 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_2_2.head()

,count
0,1468372694


In [18]:
# Overall ETH sent

query_2_3 = """
select sum(round((value/1000000000000000000),2)) as ETH_value_sent 
from  `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status is not null
"""

query_job = client.query(query_2_3)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_2_3 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_2_3.head()

,ETH_value_sent
0,5102204360.37


### <span style="color:red">3. ETH recieved</span>

In [19]:
query_3 = """
select sum(round((value/1000000000000000000),2)) as ETH_value_received
from  `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status = 1
"""

query_job = client.query(query_3)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_3 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_3.head()

,ETH_value_received
0,5062063238.35


### <span style="color:red">4. Average monthly transactions sent</span>

In [20]:
query_4 = """
select round(avg(count_transactions_sent),2) as avg_monthly_trans_sent
from
(select count(`bigquery-public-data.crypto_ethereum.transactions`.hash) as count_transactions_sent, extract(year from block_timestamp) as year, extract(month from block_timestamp) as month
from  `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status is not null
group by extract(year from block_timestamp), extract(month from block_timestamp)) T


"""

query_job = client.query(query_4)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_4 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_4.head()

,avg_monthly_trans_sent
0,27056405.16


### <span style="color:red">5. Average monthly transactions recieved</span>

In [21]:
query_5 = """
select round(avg(count_transactions_received),2) as avg_monthly_trans_received
from
(select count(`bigquery-public-data.crypto_ethereum.transactions`.hash) as count_transactions_received, extract(year from block_timestamp) as year, extract(month from block_timestamp) as month
from  `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status = 1
group by extract(year from block_timestamp), extract(month from block_timestamp)) T


"""

query_job = client.query(query_5)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_5 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_5.head()

,avg_monthly_trans_received
0,26221597.52


### <span style="color:red">6. Average monthly ETH sent</span>

In [22]:
query_6 = """
select round(avg(sum_ETH_sent),2) as avg_monthly_ETH_sent
from
(select sum(round((value/1000000000000000000),2)) as sum_ETH_sent, extract(year from block_timestamp) as year, extract(month from block_timestamp) as month
from  `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status is not null
group by extract(year from block_timestamp), extract(month from block_timestamp)) T

"""

query_job = client.query(query_6)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_6 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_6.head()

,avg_monthly_ETH_sent
0,91111408.09


### <span style="color:red">7. Average monthly ETH received</span>

In [23]:
query_7 = """
select round(avg(sum_ETH_received),2) as avg_monthly_ETH_received
from
(select sum(round((value/1000000000000000000),2)) as sum_ETH_received, extract(year from block_timestamp) as year, extract(month from block_timestamp) as month
from  `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status = 1
group by extract(year from block_timestamp), extract(month from block_timestamp)) T

"""

query_job = client.query(query_7)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_7 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_7.head()

,avg_monthly_ETH_received
0,90394614.64


### <span style="color:red">8. Average time between transactions sent</span>

In [25]:
query_8 = """
select round(avg(d),4) as avg_second_between_trans_sent
from
(select  TIMESTAMP_DIFF(lead(block_timestamp, 1) over(order by block_timestamp),block_timestamp, second) as d
from `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status is not null)

"""

query_job = client.query(query_8)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_8 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_8.head()

,avg_second_between_trans_sent
0,0.0958


### <span style="color:red">9. Average time between transactions received</span>

In [26]:
query_9 = """
select round(avg(d),4) as avg_second_between_trans_received
from
(select  TIMESTAMP_DIFF(lead(block_timestamp, 1) over(order by block_timestamp),block_timestamp, second) as d
from `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status = 1)

"""

query_job = client.query(query_9)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_9 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_9.head()

,avg_second_between_trans_received
0,0.0989


### <span style="color:red">10. STD of time between transactions sent</span>

In [29]:
query_10 = """
select round(stddev(d),4) as STD_second_between_trans_sent
from
(select  TIMESTAMP_DIFF(lead(block_timestamp, 1) over(order by block_timestamp),block_timestamp, second) as d
from `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status is not null)

"""

query_job = client.query(query_10)
iterator = query_job.result(timeout=100)
rows = list(iterator)
df_10 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_10.head()

,STD_second_between_trans_sent
0,1.5741


### <span style="color:red">11. STD of time between transactions received</span>

In [30]:
query_11 = """
select round(stddev(d),4) as STD_second_between_trans_received
from
(select  TIMESTAMP_DIFF(lead(block_timestamp, 1) over(order by block_timestamp),block_timestamp, second) as d
from `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status = 1)

"""

query_job = client.query(query_11)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_11 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_11.head()

,STD_second_between_trans_received
0,1.5989


### <span style="color:red">12. Active months</span>

In [33]:
# The most 3 active months

query_12 = """
select months as the_most_3_active_months
from
(select count(`bigquery-public-data.crypto_ethereum.transactions`.hash) as count_transactions_received, extract(month from block_timestamp) as months
from  `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status = 1
group by extract(month from block_timestamp)) T
order by count_transactions_received desc
limit 3

"""

query_job = client.query(query_12)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_12 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_12

,the_most_3_active_months
0,5
1,1
2,4


### <span style="color:red">13. ETH balance</span>

In [36]:
# ETH balance by addresses (the biggest 10 balances)

query_13 = """
select to_address, sum(round((value/1000000000000000000),2)) as ETH_balance
from  `bigquery-public-data.crypto_ethereum.transactions`
where receipt_status = 1
group by to_address
order by ETH_balance desc
limit 10

"""

query_job = client.query(query_13)
iterator = query_job.result(timeout=50)
rows = list(iterator)
df_13 = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df_13

,to_address,ETH_balance
0,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,160873160.44
1,0x876eabf441b2ee5b5b0554fd502a8e0600950cfa,72218720.13
2,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,69848551.61
3,0xfa52274dd61e1643d2205169732f29114bc240b3,43872142.07
4,0x28c6c06298d514db089934071355e5743bf21d60,43838250.98
5,0xd551234ae421e3bcba99a0da6d736074f22192ff,31616623.32
6,0x0681d8db095565fe8a346fa0277bffde9c0edbbf,31573407.21
7,0x564286362092d8e7936f0549571a803b203aaced,31083367.28
8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,29944896.36
9,0xa7efae728d2936e78bda97dc267687568dd593f3,27139962.49
